Script para realizar la predicción a 15 días utilizando una red de capa única de 512 neuronas con variables de entrada Bitcoin, EOS, Qtum, OmiseGo y ZCash.

In [6]:
# Manipulación de datos
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt # Plots
import matplotlib.gridspec as gridspec
params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

# Keras
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.layers.core import Dense,Activation,Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import time
import warnings
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from math import sqrt
from scipy import stats
from statsmodels.tsa.api import ExponentialSmoothing

#Manejo de archivos
import os
import tempfile
import shutil
import json

import json

import DataManager as dm

colores_test = ['r','g','b','c']
colores_futuro = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
criptomonedas_correlacionadas = ['eos', 'qtum', 'omisego', 'zcash']  

# Conf
test_size = 30
colores = ['r','g','b','c']

# Convertir el conjunto de datos como dos conjuntos x e y
def create_dataset(dataset,col_target,f):
    dataX, dataY = [], []
    for i in range(len(dataset)-f):  
        dataX.append(dataset[i,:])  #precio (BHC,ETH,LCT,XRP,...)
        dataY.append(dataset[i+f,col_target])  #precio BTC del dia futuro f
    return np.asarray(dataX), np.asarray(dataY)

# Convertir el conjunto de datos como dos conjuntos x e y, donde x no contiene al target
def create_dataset_sintarget(dataset,col_target,f):
    dataX, dataY = [], []
    dataset_sintarget = np.delete(dataset, col_target, 1)
    for i in range(len(dataset)-f):  
        dataX.append(dataset_sintarget[i,:])  #precio (BHC,ETH,LCT,XRP,...)
        dataY.append(dataset[i+f,col_target])  #precio BTC del dia futuro f
    return np.asarray(dataX), np.asarray(dataY)

def create_batches(dataset,targets,v):
    b = []
    t = []
    for i in range(len(dataset)-v):
        batch = dataset[i:i+v+1]
        b.append(batch)
        t.append(targets[i+v])
    return np.asarray(b),np.asarray(t)

# Método para entrenar la red
def entrenamiento(dataset, maxf, target):
    prediccion = dataset[[target]][-test_size:].copy()
    v = 0  
    predicciones_f = [0 for f in range(0,maxf-1)] # Arreglo con las maxf predicciones, una por cada iteracion
    
    for f in range(1,maxf):
        # Separar el datset en x e y (predicción para el día f)
        x,y = create_dataset(np.asarray(dataset),dataset.columns.get_loc(target),f)
        y = y.reshape(-1,1)

        # Separar x,y en conjuntos de entrenamiento y testeo (test_size+v para poder hacer ventanas de timesteps después de escalar y que siempre queden 30 puntos de predicción en test)
        train_x, train_y = x[:-test_size], y[:-test_size]
        test_x, test_y = x[-test_size-v:], y[-test_size-v:]

        # Normalización: Escalar los datos entre [0,1]. Hay que escalar con respecto al training set, como si el test set no estuviera.
        scalerX = preprocessing.MinMaxScaler(feature_range = (0,1)).fit(train_x)
        train_x_scaled = scalerX.transform(train_x)
        test_x_scaled = scalerX.transform(test_x)
        scalerY = preprocessing.MinMaxScaler(feature_range = (0,1)).fit(train_y)
        train_y_scaled = scalerY.transform(train_y)
        test_y_scaled = scalerY.transform(test_y)  

        # Crear batches de v timesteps (ventana)
        input_train,target_train = create_batches(train_x_scaled,train_y_scaled,v)
        input_test,target_test = create_batches(test_x_scaled,test_y_scaled,v)

        #Construir red neuronal y entrenarla
        model = Sequential()
        model.add(LSTM(units = 2,
                       input_shape=(input_train.shape[1],input_train.shape[2]),
                       return_sequences=False, activation = 'tanh'))
        model.add(Dense(units = 1, activation = 'linear'))
        model.compile(optimizer = 'adam', loss = 'mean_squared_error')
        early_stopping_monitor = EarlyStopping(monitor='val_loss',patience=10,verbose=1) # Stop loss después de *patience* epochs consecutivos sin mejora
        history = model.fit(input_train, target_train, validation_split=0.2,
                            batch_size=128, epochs = 300, callbacks=[early_stopping_monitor],
                            shuffle=False,verbose=1)  

        # Dataframe de entrenamiento
        training_history = pd.DataFrame() 
        training_history = pd.concat([training_history, pd.DataFrame(data=history.history['loss'], columns=['f' + str(f) +'_loss'])], axis=1)
        training_history = pd.concat([training_history, pd.DataFrame(data=history.history['val_loss'], columns=['f' + str(f) + '_val_loss'])], axis=1)
        training_history.to_csv(datasetfolder + '/entrenamiento ' + 'f' + str(f) + '.csv')
        print('Entrenamiento ' + 'f' + str(f) + ' completo.')
        #entrenamientos.append(training_history)   

        #Predicción de testeo
        predicted_scaled = model.predict(input_test)
        predicted_scaled = predicted_scaled.reshape(-1,1)
        predicted = scalerY.inverse_transform(predicted_scaled)
        
        print(predicted)
        print(len(predicted))

        # Dataframe de prediccion del subset  
        prediccion['f' + str(f)] = predicted
        print("Simulación "+ 'f' + str(f) +" completa.")        
        
        # HACER LA PREDICCION PARA EL DIA F
        x = np.asarray(dataset[-1:])  
        x = scalerX.transform(x)
        input_test,target_test = create_batches(x,x,0)
        prediccionf = model.predict(input_test)
        prediccionf = prediccionf.reshape(-1,1)
        prediccionf = scalerY.inverse_transform(prediccionf)
        predicciones_f[f-1] = prediccionf.flatten()[0]
        
    # Agregar la predicción al dataframe del dataset
    prediccion = pd.merge(dataset[[target]],prediccion.drop(columns=[target]),how="left",left_index=True,right_index=True)            
    print(prediccion)
    prediccion.to_csv(datasetfolder + '/' + 'resultados.csv')
    
    # Construir dataframe con la prediccion a f dias a futuro
    predicciones_f_dataframe = pd.DataFrame(pd.date_range(start=dataset.index[-1], periods=maxf, freq='D', closed='right'),columns=['Date'])
    predicciones_f_dataframe['Prediccion'] = predicciones_f
        
    # Crear dataframe con precio real + prediccion (con NULLs). Este es el json que va a Angular
    dfFinal = pd.merge(prediccion[[target]], predicciones_f_dataframe, how="outer",left_index=True,right_index=True)
    
    # Holt-Winters lineal (conjunto de entrenamiento fijo)
    ciclos = 45; alfa = 0.1; beta = 0.1
    prediccionesHW = pd.DataFrame()
    datasetHW = dataset[[target]].copy()
    testHW = datasetHW[-test_size:]
    # Entrenar y obtener RMSE del entrenamiento
    testHW['holtwinters'] = np.nan
    trainingHW = datasetHW[0:-test_size]
    hw_lineal = ExponentialSmoothing(np.asarray(trainingHW), seasonal_periods=ciclos, trend='add', seasonal='add').fit(smoothing_level = alfa,smoothing_slope = beta)
    testHW['holtwinters'] = hw_lineal.forecast(test_size)
    testHW['holtwinters'] = testHW['holtwinters'] .interpolate().ffill().fillna(0)
    prediccionesHW = prediccionesHW.append(testHW)
    prediccionesHW = pd.merge(dataset.copy()[[target]],prediccionesHW.drop(columns=[target]), how="left",left_index=True,right_index=True)
    subset_moneda = prediccionesHW[[target]][-test_size:]
    subset_holtwinters = prediccionesHW[['holtwinters']][-test_size:]              
    #RMSE
    rmseHW = sqrt(mean_squared_error(subset_moneda, subset_holtwinters))

    # Realizar la predicción a maxF días
    trainingHW = dataset[[target]].copy()
    hw_lineal = ExponentialSmoothing(np.asarray(trainingHW), seasonal_periods=ciclos, trend='add', seasonal='add').fit(smoothing_level = alfa,smoothing_slope = beta)
    holtwinters = hw_lineal.forecast(maxf-1)

    # Agregar holwinters al dataframe con la prediccion a f dias a futuro
    predicciones_f_dataframe['holtwinters'] = holtwinters   
    
    predicciones_f_dataframe['Date'] = pd.to_datetime(predicciones_f_dataframe['Date'])
    predicciones_f_dataframe.set_index('Date', inplace=True)
    predicciones_f_dataframe.to_csv(datasetfolder + '/' + 'predicciones_f.csv')
    predicciones_f_dataframe.to_json(datasetfolder + '/predicciones_f.json',orient="split")
    
    # Crear dataframe con precio real + prediccion (con NULLs). Este es el json que va a Angular
    dfFinal = pd.merge(data[[target]], predicciones_f_dataframe, how="outer",left_index=True,right_index=True)
    dfFinal.to_csv(datasetfolder + '/' + 'toAngular.csv')
    dfFinal.to_json(datasetfolder + '/toAngular.json',orient="split")
    
    return (prediccion, predicciones_f_dataframe, dfFinal, rmseHW)


# Construir dataframes con RMSE y epochs por test
def postprocesamiento(prediccion, name, name_parametro, referencia = []):
    rmse_completo_y = []
    rmse_completo_x = []
    arreglo_nepochs = []
    for simulacion_i in prediccion.columns[1:]:
                    dataset_name = simulacion_i; i=1
                    prediccionxdataset = pd.DataFrame()
                    arreglo_nepochs_simulacion_i = [simulacion_i]
                    subset = prediccion
                    subset_moneda = subset[subset.columns[0]]
                    subset_prediccion = subset[[simulacion_i]][-test_size:]               
                                                
                    #Cada entrenamiento                        
                    entrenamiento = pd.read_csv(datasetfolder + '/entrenamiento ' + dataset_name + '.csv',index_col =0)
                    arreglo_nepochs_simulacion_i.append(entrenamiento.shape[0]) # Obtener cantidad de epochs
                                              
                    #RMSE
                    rmse = sqrt(mean_squared_error(subset_moneda[-test_size:], subset_prediccion))
                    rmse_completo_y.append(rmse)
                    rmse_completo_x.append(simulacion_i)
                    
                    arreglo_nepochs.append(arreglo_nepochs_simulacion_i) #Agregar la cantidad de epochs para la simulacion i
                    
                    i+=1
                    
    # Construir un csv con todos los RMSE
    arreglo_rmse = []; i=0
    for simulacion_i in prediccion.columns[1:]:
        arreglo_rmse.append([simulacion_i] + [rmse_completo_y[i]]); i+=1
    dataframe_rmse = pd.DataFrame(arreglo_rmse, columns=['Simulacion','RMSE'])
    dataframe_rmse.set_index('Simulacion',inplace=True)
    dataframe_rmse.to_csv(datasetfolder + '/post/' + name + ' rmses.csv')
    dataframe_rmse.to_json(datasetfolder + '/post/' + name + ' rmses.json',orient="columns")
    print('RMSEs guardados: ' + datasetfolder + '/post/' + name + ' rmses.csv')
    
    # Construir un csv con los epochs 
    dataframe_epochs = pd.DataFrame(arreglo_nepochs, columns=['Simulacion','Epochs'])
    dataframe_epochs.set_index('Simulacion',inplace=True)
    dataframe_epochs.to_csv(datasetfolder + '/post/' + name + ' epochs.csv')
    print('Epochs guardados: ' + datasetfolder + '/post/' + name + ' rmses.csv')
        
    # Graficar RMSE en barras
    width = 1/1.5
    plt.figure(figsize=(25,8))
    barplot = plt.bar(rmse_completo_x, rmse_completo_y, width)
    col = 0;
    for rect in barplot: # Valores en las barras
        height = rect.get_height()
        plt.text(rect.get_x() + rect.get_width()/2.0, height, '% 6.2f' % height,fontsize=22, ha='center',va='bottom',rotation=90)
        rect.set_color(colores_test[col])
        col = (col+1)%4
    plt.xticks(rotation=60, ha="right")
    plt.ylim(ymax=max(rmse_completo_y)*1.2)
    plt.ylabel('Error', fontsize=18)
    plt.title("RMSE", fontsize=22)
    plt.savefig(datasetfolder + '/post/' + name + ' RMSE barra.png', bbox_inches='tight')  
    
    #Graficar RMSE por variación del parámetro
    plt.figure(figsize=(20,8)); col = 0
    for i in range(0,len(dataframe_rmse.columns)):
        plt.plot(dataframe_rmse.index, dataframe_rmse[dataframe_rmse.columns[i]], label=dataframe_rmse.columns[i], color=colores_test[col]);  col = (col+1)%4
    for r in referencia:
        plt.axhline(r,linestyle='--', color=colores_test[col], alpha=0.6); col=(col+1)%4
    plt.legend(loc='upper left')
    plt.grid(color='grey', linestyle='-', linewidth=0.5)
    plt.xticks(rotation=20, ha="right")
    plt.ylabel('Error', fontsize=18)
    plt.title("RMSE vs. " + name_parametro,fontsize=20)
    plt.savefig(datasetfolder + '/post/' + name + ' RMSE lineal.png', bbox_inches='tight') 
    plt.show()
    
    #Graficar EPOCHS por variación del parámetro
    plt.figure(figsize=(20,8)); col = 0
    for i in range(0,len(dataframe_epochs.columns)):
        plt.plot(dataframe_epochs.index, dataframe_epochs[dataframe_epochs.columns[i]], label=dataframe_epochs.columns[i], color=colores_test[col]);  col = (col+1)%3
    plt.legend(loc='upper left')
    plt.grid(color='grey', linestyle='-', linewidth=0.5)
    plt.xticks(rotation=20, ha="right")
    plt.ylabel('Epochs', fontsize=18)
    plt.title("Epochs vs. " + name_parametro,fontsize=20)
    plt.savefig(datasetfolder + '/post/' + name + ' epochs lineal.png', bbox_inches='tight') 
    plt.show()  
    
    return (dataframe_rmse,dataframe_epochs)

# Graficar la simulacion i para la prediccion 
def graficarSimulacion(prediccion, dataframe_rmse, dataframe_epochs, name, name_parametro):
        for dataset_name in prediccion.columns[1:]:       
                    #Visualización de resultados
                    fig = plt.figure(figsize=(25,10*2))
                    gs = gridspec.GridSpec(2, 5, wspace=0.3, hspace=0.4)
                    i = 1

                    #Todas los subsets para un dataset
                    axPrediccionCompleta = plt.subplot(gs[0, 0:])
                    axPrediccionCompleta.plot(prediccion.index, prediccion[prediccion.columns[0]], color='dimgray', label="Precio Real")  
                    axPrediccionCompleta.set_ylabel('Precio (USD)', fontsize=18)
                    plt.xticks(rotation=30, ha="right")
                    axPrediccionCompleta.set_title("Predicciones " + dataset_name,fontsize=22)
                    axPrediccionCompleta.grid(color='grey', linestyle='-', linewidth=0.5) 
                    rmse_general = dataframe_rmse.loc[dataset_name, 'RMSE'] # Obtener el RMSE para la prediccion
                    axPrediccionCompleta.text(0.05, 0.1, 'RMSE ' + str('% 6.2f' % rmse_general), 
                                            horizontalalignment='left',verticalalignment='center', 
                                            transform=axPrediccionCompleta.transAxes, fontsize=17, bbox=dict(facecolor='red', alpha=0.5))
                  
                    n_test = 0
                    prediccionxdataset = pd.DataFrame()
                    subset = prediccion
                    subset_moneda = subset[subset.columns[0]]
                    subset_prediccion = subset[[dataset_name]][-test_size:]
                    prediccionxdataset = prediccionxdataset.append(
                            pd.concat([subset_moneda[-test_size:], subset_prediccion], axis=1))
                        
                    #Graficar la serie completa
                    axPrediccionCompleta.plot(subset_prediccion.index, subset_prediccion, color=colores_test[n_test], label='Prediccion (' + str(n_test+1) + ')')
                    axPrediccionCompleta.axvspan(subset_prediccion.index[0], subset_prediccion.index[-1], facecolor=colores_test[n_test], alpha=0.1)  
                    axPrediccionCompleta.legend(loc='upper left')                        
                                                
                    #Cada entrenamiento                        
                    entrenamiento = pd.read_csv(datasetfolder + '/entrenamiento ' + dataset_name + '.csv',index_col =0)
                    axTraining = plt.subplot(gs[i, 0])
                    axTraining.plot(entrenamiento[entrenamiento.columns[0]], label='loss')
                    axTraining.plot(entrenamiento[entrenamiento.columns[1]], label='val_loss')
                    axTraining.set_xlabel('Epochs', fontsize=18)
                    axTraining.set_ylabel('Loss', fontsize=18)
                    axTraining.legend(loc='upper right')
                    axTraining.set_title("Entrenamiento", fontsize=22)
                                              
                    #Cada prediccion   
                    axPrediccion = plt.subplot(gs[i, 1:])
                    axPrediccion.plot(subset_moneda.index[-45:], subset_moneda[-45:], color='dimgray', label="Precio Real")
                    axPrediccion.plot(subset_prediccion.index, subset_prediccion, color=colores_test[n_test],  label='Prediccion ' + dataset_name)
                    axPrediccion.axvspan(subset_prediccion.index[0], subset_prediccion.index[-1], facecolor=colores_test[n_test], alpha=0.1)
                    axPrediccion.set_ylabel('Precio (USD)', fontsize=18)
                    plt.xticks(rotation=30, ha="right")
                    axPrediccion.legend(loc='upper left')
                    axPrediccion.set_title("Predicciones",fontsize=22)
                    axPrediccion.grid(color='grey', linestyle='-', linewidth=0.5)                         
                    rmse_i = dataframe_rmse.loc[dataset_name, 'RMSE'] # Obtener el RMSE para la prediccion
                    axPrediccion.text(0.05, 0.1, 'RMSE ' + str('% 6.2f' % rmse_i), 
                                            horizontalalignment='left',verticalalignment='center', 
                                            transform=axPrediccion.transAxes, fontsize=17, bbox=dict(facecolor='red', alpha=0.5))
                    i+=1
                    
                    plt.savefig(datasetfolder + '/post/' + name + ' Curvas ' + dataset_name + '.png', bbox_inches='tight') 

target_list = ['bitcoin', 'ethereum', 'ripple', 'bitcoin-cash']
PrediccionMeta = {}
for target in target_list:
    # Crear carpeta donde se almacenarán las predicciones  
    datasetfolder = "script" + "/"  + target
    if not os.path.exists(datasetfolder):
        os.makedirs(datasetfolder) # Crear carpeta de predicciones
    postfolder = datasetfolder + '/post'
    if not os.path.exists(postfolder):
        os.makedirs(postfolder) # Crear carpeta de postprocesamiento

    if True:
        targets_crypto = [target] + criptomonedas_correlacionadas
        data = dm.get_crypto(targets_crypto,fecha_comienzo='2015-01-01',fecha_fin='2018-08-31')
        data = data.interpolate().ffill().fillna(0) #Completar campos faltantes
        data.to_csv(datasetfolder + '/data ' + target +'.csv')

    data = pd.read_csv(datasetfolder + '/data ' + target + '.csv', usecols=['Date', 'Close ' + target, 'Close eos', 'Close qtum', 'Close omisego', 'Close zcash'])
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    data[['Close ' + target]].to_json(datasetfolder + '/data '+ target +'.json',orient="split")

    #Graficar la serie completa
    plt.figure(figsize=(20,8))
    for i in range(0,len(data.columns)):
        plt.plot(data.index, data[data.columns[i]], label=data.columns[i])
    plt.legend(loc='upper left')
    plt.title("Datos",fontsize=20)
    plt.show()

    # ENTRENAR Y OBTENER PREDICCIONES
    prediccion_entrenamiento, predicciones_f, dfFinal, rmseHW = entrenamiento(data.copy(), 4, 'Close ' + target)

    dataframe_rmse, dataframe_epochs = postprocesamiento(prediccion_entrenamiento,  name='Predicciones entrenamiento', name_parametro='f')
    graficarSimulacion(prediccion_entrenamiento, dataframe_rmse, dataframe_epochs, name='Predicciones entrenamiento', name_parametro='f')

    # Guardar datos sobre la prediccionF para los gráficos 
    minP = predicciones_f[['Prediccion']].values.min()
    maxP = predicciones_f[['Prediccion']].values.max()
    # Calcular la performance: pendiente de la regresion lineal sobre la curva de prediccion.
    xi = np.arange(0,len(predicciones_f.index))
    slope, intercept, r_value, p_value, std_err = stats.linregress(xi,predicciones_f[['Prediccion']].values.flatten())
    line = slope*xi+intercept
    # Calcular la volatilidad de la prediccion: deviación estándar de la curva.
    volatilidad = np.std(predicciones_f[['Prediccion']].values)

    targetMeta = {
        'RMSE': dataframe_rmse.values.flatten()[0],
        'MinimoPrecio': minP,
        'MaximoPrecio': maxP,
        'Performance': slope,
        'Volatilidad':volatilidad,
        'VolatilidadRelativa': 100*volatilidad/abs((maxP-minP)/2),
        'RMSEHW': rmseHW
    }
    PrediccionMeta[target] = targetMeta
    
# Agregar información general al archivo meta    
MaxPerformanceMoneda = MinPerformanceMoneda = MaxVolatilidadMoneda = MinVolatilidadMoneda = target_list[0]
MaxPerformance = PrediccionMeta[target_list[0]]['Performance']
MinPerformance = PrediccionMeta[target_list[0]]['Performance']
MaxVolatilidad = PrediccionMeta[target_list[0]]['Volatilidad']
MinVolatilidad = PrediccionMeta[target_list[0]]['Volatilidad']

for p in PrediccionMeta:
    if(PrediccionMeta[p]['Performance'] > MaxPerformance):
        MaxPerformanceMoneda = p
        MaxPerformance = PrediccionMeta[p]['Performance']
    if(PrediccionMeta[p]['Performance'] < MinPerformance):
        MinPerformanceMoneda = p
        MinPerformance = PrediccionMeta[p]['Performance']
    if(PrediccionMeta[p]['Volatilidad'] > MaxVolatilidad):
        MaxVolatilidadMoneda = p
        MaxVolatilidad = PrediccionMeta[p]['Volatilidad']
    if(PrediccionMeta[p]['Volatilidad'] < MinVolatilidad):
        MinVolatilidadMoneda = p
        MinVolatilidad = PrediccionMeta[p]['Volatilidad']
        
PrediccionMeta['General'] = {
    'MaxPerformance': {
        'Moneda': MaxPerformanceMoneda,
        'Performance': MaxPerformance
    },
    'MinPerformance': {
        'Moneda': MinPerformanceMoneda,
        'Performance': MinPerformance
    },
    'MaxVolatilidad': {
        'Moneda': MaxVolatilidadMoneda,
        'Volatilidad': MaxVolatilidad
    },
    'MinVolatilidad': {
        'Moneda': MinVolatilidadMoneda,
        'Volatilidad': MinVolatilidad
    }
}
print(PrediccionMeta)
with open("script" + '/PrediccionMeta.json', 'w') as outfile:
    json.dump(PrediccionMeta, outfile)

c:\users\leandro\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Criptomoneda procesada: bitcoin
Criptomoneda procesada: eos
Criptomoneda procesada: qtum
Criptomoneda procesada: omisego
Criptomoneda procesada: zcash


<Figure size 2000x800 with 1 Axes>

Train on 1046 samples, validate on 262 samples
Epoch 1/300
1046/1046 [==============================] - 4s 3ms/step - loss: 0.0020 - val_loss: 0.0928
Epoch 2/300
1046/1046 [==============================] - 0s 25us/step - loss: 0.0016 - val_loss: 0.0871
Epoch 3/300
1046/1046 [==============================] - 0s 39us/step - loss: 0.0015 - val_loss: 0.0820
Epoch 4/300
1046/1046 [==============================] - 0s 20us/step - loss: 0.0014 - val_loss: 0.0776
Epoch 5/300
1046/1046 [==============================] - 0s 30us/step - loss: 0.0014 - val_loss: 0.0738
Epoch 6/300
1046/1046 [==============================] - 0s 30us/step - loss: 0.0014 - val_loss: 0.0705
Epoch 7/300
1046/1046 [==============================] - 0s 30us/step - loss: 0.0014 - val_loss: 0.0677
Epoch 8/300
1046/1046 [==============================] - 0s 42us/step - loss: 0.0014 - val_loss: 0.0652
Epoch 9/300
1046/1046 [==============================] - 0s 31us/step - loss: 0.0014 - val_loss: 0.0630
Epoch 10/300
1046/

1046/1046 [==============================] - 0s 46us/step - loss: 5.0076e-04 - val_loss: 0.0146
Epoch 77/300
1046/1046 [==============================] - 0s 50us/step - loss: 5.0186e-04 - val_loss: 0.0146
Epoch 78/300
1046/1046 [==============================] - 0s 36us/step - loss: 5.0292e-04 - val_loss: 0.0145
Epoch 79/300
1046/1046 [==============================] - 0s 30us/step - loss: 5.0393e-04 - val_loss: 0.0145
Epoch 80/300
1046/1046 [==============================] - 0s 23us/step - loss: 5.0490e-04 - val_loss: 0.0144
Epoch 81/300
1046/1046 [==============================] - 0s 30us/step - loss: 5.0581e-04 - val_loss: 0.0144
Epoch 82/300
1046/1046 [==============================] - 0s 59us/step - loss: 5.0667e-04 - val_loss: 0.0143
Epoch 83/300
1046/1046 [==============================] - 0s 42us/step - loss: 5.0747e-04 - val_loss: 0.0143
Epoch 84/300
1046/1046 [==============================] - 0s 30us/step - loss: 5.0821e-04 - val_loss: 0.0142
Epoch 85/300
1046/1046 [========

...

RMSEs guardados: script/bitcoin-cash/post/Predicciones entrenamiento rmses.csv
Epochs guardados: script/bitcoin-cash/post/Predicciones entrenamiento rmses.csv


<Figure size 2500x800 with 1 Axes>

<Figure size 2000x800 with 1 Axes>

<Figure size 2000x800 with 1 Axes>

{'bitcoin': {'RMSE': 500.1889991028144, 'MinimoPrecio': 5474.89892578125, 'MaximoPrecio': 6584.5166015625, 'Performance': -539.033447265625, 'Volatilidad': 515.8030488996876, 'VolatilidadRelativa': 92.96950835548388, 'RMSEHW': 2687.4866430026236}, 'ethereum': {'RMSE': 28.834454912999934, 'MinimoPrecio': 245.43917846679688, 'MaximoPrecio': 300.52642822265625, 'Performance': -27.543624877929688, 'Volatilidad': 22.505054298867254, 'VolatilidadRelativa': 81.70694452384963, 'RMSEHW': 148.79856371127138}, 'ripple': {'RMSE': 0.055534688732142394, 'MinimoPrecio': 0.28514406085014343, 'MaximoPrecio': 0.32998883724212646, 'Performance': 0.019141942262649536, 'Volatilidad': 0.01977584128307318, 'VolatilidadRelativa': 88.19685534928226, 'RMSEHW': 0.17455419502911665}, 'bitcoin-cash': {'RMSE': 108.9210885491567, 'MinimoPrecio': 482.56268310546875, 'MaximoPrecio': 713.5812377929688, 'Performance': 6.070953369140625, 'Volatilidad': 106.15710868887197, 'VolatilidadRelativa': 91.90353461649109, 'RMSEHW